In [1]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
import pandas as pd
from tqdm import tqdm


In [3]:
# Load the dataset
df = pd.read_csv('most_toxic_data.csv').head(1000)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer(df['reference'].tolist(), max_length=128, padding=True, truncation=True, return_tensors="pt")

In [4]:
# Dataset class
class DetoxificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)

# Prepare dataset and dataloader
dataset = DetoxificationDataset(tokens, df['trn_tox'])
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [5]:
# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# Define optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)
epochs = 5


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
# Assuming 'epochs' is defined and you have a 'dataloader' for your data
loss_values = []  # List to store loss values

for epoch in tqdm(range(epochs), desc="Epoch"):
    total_loss = 0  # Variable to store the epoch loss

    for batch in tqdm(dataloader, desc="Iteration"):
        optimizer.zero_grad()

        # Unpack the batch contents and feed them into the model
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass and compute loss
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()

        # Update total loss
        total_loss += loss.item()

    # Calculate the average loss over the entire epoch
    avg_epoch_loss = total_loss / len(dataloader)
    print(f"Average Loss at Epoch {epoch}: {avg_epoch_loss}")

    # Store the average loss value for plotting or analysis later
    loss_values.append(avg_epoch_loss)

# After training loop, you might want to save the model


Iteration:   0%|          | 0/63 [00:00<?, ?it/s]<ipython-input-4-a53a1f3cd555>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

Epoch:  20%|██        | 1/5 [10:24<41:36, 624.24s/it]

Average Loss at Epoch 0: 0.07926585171963015



Epoch:  40%|████      | 2/5 [20:40<30:57, 619.28s/it]

Average Loss at Epoch 1: 0.0042768856732263456



Epoch:  60%|██████    | 3/5 [30:48<20:28, 614.39s/it]

Average Loss at Epoch 2: 0.0018442948441213323



Epoch:  80%|████████  | 4/5 [41:00<10:13, 613.30s/it]

Average Loss at Epoch 3: 0.0010203922782758518



Epoch: 100%|██████████| 5/5 [51:11<00:00, 614.32s/it]

Average Loss at Epoch 4: 0.0006582681856502498


In [7]:
# Save the model's state_dict
model.save_pretrained('bert_model')
tokenizer.save_pretrained('bert_model')

('bert_model/tokenizer_config.json',
 'bert_model/special_tokens_map.json',
 'bert_model/vocab.txt',
 'bert_model/added_tokens.json')

In [8]:
!zip -r bert_model.zip bert_model

  adding: bert_model/ (stored 0%)
  adding: bert_model/vocab.txt (deflated 53%)
  adding: bert_model/tokenizer_config.json (deflated 75%)
  adding: bert_model/special_tokens_map.json (deflated 42%)
  adding: bert_model/config.json (deflated 49%)
  adding: bert_model/model.safetensors (deflated 7%)


In [13]:
def predict(toxicity_classifier, tokenizer, text):
    # Prepare the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Get model predictions
    with torch.no_grad():
        outputs = toxicity_classifier(input_ids, attention_mask=attention_mask)

    logits = outputs.logits.squeeze()  # Squeeze the logits to remove any extra dimensions

    # Apply softmax if there are multiple classes (logits would be a 1D Tensor with more than one element)
    if logits.numel() > 1:  # More than one element means multi-class classification
        probabilities = torch.softmax(logits, dim=0)
        predicted_class = torch.argmax(probabilities).item()
        return probabilities, predicted_class
    # Apply sigmoid if it's binary classification (logits would be a single-element tensor)
    else:  # Single element means binary classification
        probability = torch.sigmoid(logits).item()
        return probability

In [17]:
example_text = "I like that shit."
prediction = predict(model, tokenizer, example_text)

# Process the prediction
if isinstance(prediction, tuple):  # Multi-class classification
    probabilities, predicted_class = prediction
    print(f"Probabilities: {probabilities[0].item()}")
    print(f"Predicted toxic: {'Yes' if predicted_class == 0 else 'No'}")


Probabilities: 0.9992675185203552
Predicted toxic: Yes
